전국 평균 평당 분양가격(2013년 9월부터 2015년 8월까지).csv (지역명, 평당분양가격, 연도, 월)
- 지역명2: 지역명필드는 라벨인코딩하여 추가
- 평당분양가격n: 평당분양가격의 정규화 스케일 조정하여 추가
- 평당분양가격s: 평당분양가격의 표준화 스케일 조정하여 추가
- 연도n: 연도의 정규화 스케일 조정하여 추가
- 연도s: 연도의 표준화 스케일 조정하여 추가
- 월n: 월의 정규화 스케일 조정하여 추가
- 월s: 월의 표준화 스케일 조정하여 추가


In [3]:
import pandas as pd

import numpy as np

In [5]:
csvdata = pd.read_csv(r'D:/ai/Downloads/shareData/전국 평균 평당 분양가격(2013년 9월부터 2015년 8월까지).csv', 
                      encoding = 'cp949')
csvdata.head(1)

,지역,2013년12월,2014년1월,2014년2월,2014년3월,2014년4월,2014년5월,2014년6월,2014년7월,2014년8월,...,2014년11월,2014년12월,2015년1월,2015년2월,2015년3월,2015년4월,2015년5월,2015년6월,2015년7월,2015년8월
0,서울,18189,17925,17925,18016,18098,19446,18867,18742,19274,...,20242,20269,20670,20670,19415,18842,18367,18374,18152,18443


In [8]:
df = csvdata.melt(id_vars = '지역',
                  var_name = '연도월',
                  value_name = '평당분양가격')
df

,지역,연도월,평당분양가격
0,서울,2013년12월,18189
1,부산,2013년12월,8111
2,대구,2013년12월,8080
3,인천,2013년12월,10204
4,광주,2013년12월,6098
...,...,...,...
352,전북,2015년8월,6580
353,전남,2015년8월,6289
354,경북,2015년8월,7037
355,경남,2015년8월,7665


In [9]:
df['연도'] = df['연도월'].apply(lambda x: int(x.split('년')[0]))
df['월']   = df['연도월'].apply(lambda x: int(x.split('년')[1].replace('월', '')))

In [11]:
df.drop('연도월', 
        axis = 1,
        inplace = True)
df = df.rename(columns = {'지역':'지역명'})
df.head(1)

,지역명,평당분양가격,연도,월
0,서울,18189,2013,12


In [82]:
from sklearn.preprocessing import LabelEncoder
city = np.array(df['지역명']).reshape(-1, 1)
df['지역명2'] = le.fit_transform(city)
df

C:\Users\4545\anaconda3\lib\site-packages\sklearn\preprocessing\_label.py:116: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


,지역명,평당분양가격,연도,월,지역명2,평당분양가격s,평당분양가격n
0,서울,18189,2013,12,8,0.835270,3.297226
1,부산,8111,2013,12,7,0.166124,-0.113291
2,대구,8080,2013,12,5,0.164066,-0.123782
3,인천,10204,2013,12,11,0.305093,0.595005
4,광주,6098,2013,12,4,0.032468,-0.794515
...,...,...,...,...,...,...,...
352,전북,6580,2015,8,13,0.064471,-0.631400
353,전남,6289,2015,8,12,0.045150,-0.729878
354,경북,7037,2015,8,3,0.094814,-0.476746
355,경남,7665,2015,8,2,0.136512,-0.264223


In [38]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler

In [74]:
price = np.array(df['평당분양가격']).reshape(-1, 1)
scaler_price = MinMaxScaler()
scaler_price.fit(price)
df['평당분양가격s'] = scaler_price.transform(price)

,지역명,평당분양가격,연도,월,지역명2,평당분양가격s
0,서울,18189,2013,12,<bound method LabelEncoder.fit_transform of La...,0.835270
1,부산,8111,2013,12,<bound method LabelEncoder.fit_transform of La...,0.166124
2,대구,8080,2013,12,<bound method LabelEncoder.fit_transform of La...,0.164066
3,인천,10204,2013,12,<bound method LabelEncoder.fit_transform of La...,0.305093
4,광주,6098,2013,12,<bound method LabelEncoder.fit_transform of La...,0.032468
...,...,...,...,...,...,...
352,전북,6580,2015,8,<bound method LabelEncoder.fit_transform of La...,0.064471
353,전남,6289,2015,8,<bound method LabelEncoder.fit_transform of La...,0.045150
354,경북,7037,2015,8,<bound method LabelEncoder.fit_transform of La...,0.094814
355,경남,7665,2015,8,<bound method LabelEncoder.fit_transform of La...,0.136512


In [83]:
scaler_price = StandardScaler()
scaler_price.fit(price)
df['평당분양가격n'] = scaler_price.transform(price)
df

,지역명,평당분양가격,연도,월,지역명2,평당분양가격s,평당분양가격n
0,서울,18189,2013,12,8,0.835270,3.297226
1,부산,8111,2013,12,7,0.166124,-0.113291
2,대구,8080,2013,12,5,0.164066,-0.123782
3,인천,10204,2013,12,11,0.305093,0.595005
4,광주,6098,2013,12,4,0.032468,-0.794515
...,...,...,...,...,...,...,...
352,전북,6580,2015,8,13,0.064471,-0.631400
353,전남,6289,2015,8,12,0.045150,-0.729878
354,경북,7037,2015,8,3,0.094814,-0.476746
355,경남,7665,2015,8,2,0.136512,-0.264223


In [85]:
year = np.array(df['연도']).reshape(-1, 1)
scaler_year = MinMaxScaler()
scaler_year.fit(year)
df['연도s'] = scaler_year.transform(year)
df

,지역명,평당분양가격,연도,월,지역명2,평당분양가격s,평당분양가격n,연도s
0,서울,18189,2013,12,8,0.835270,3.297226,0.0
1,부산,8111,2013,12,7,0.166124,-0.113291,0.0
2,대구,8080,2013,12,5,0.164066,-0.123782,0.0
3,인천,10204,2013,12,11,0.305093,0.595005,0.0
4,광주,6098,2013,12,4,0.032468,-0.794515,0.0
...,...,...,...,...,...,...,...,...
352,전북,6580,2015,8,13,0.064471,-0.631400,1.0
353,전남,6289,2015,8,12,0.045150,-0.729878,1.0
354,경북,7037,2015,8,3,0.094814,-0.476746,1.0
355,경남,7665,2015,8,2,0.136512,-0.264223,1.0


In [86]:
scaler_year = StandardScaler()
scaler_year.fit(year)
df['연도n'] = scaler_year.transform(year)
df

,지역명,평당분양가격,연도,월,지역명2,평당분양가격s,평당분양가격n,연도s,연도n
0,서울,18189,2013,12,8,0.835270,3.297226,0.0,-2.366432
1,부산,8111,2013,12,7,0.166124,-0.113291,0.0,-2.366432
2,대구,8080,2013,12,5,0.164066,-0.123782,0.0,-2.366432
3,인천,10204,2013,12,11,0.305093,0.595005,0.0,-2.366432
4,광주,6098,2013,12,4,0.032468,-0.794515,0.0,-2.366432
...,...,...,...,...,...,...,...,...,...
352,전북,6580,2015,8,13,0.064471,-0.631400,1.0,1.183216
353,전남,6289,2015,8,12,0.045150,-0.729878,1.0,1.183216
354,경북,7037,2015,8,3,0.094814,-0.476746,1.0,1.183216
355,경남,7665,2015,8,2,0.136512,-0.264223,1.0,1.183216


In [87]:
month = np.array(df['월']).reshape(-1, 1)
scaler_month = MinMaxScaler()
scaler_month.fit(month)
df['월s'] = scaler_month.transform(month)
df

,지역명,평당분양가격,연도,월,지역명2,평당분양가격s,평당분양가격n,연도s,연도n,월s
0,서울,18189,2013,12,8,0.835270,3.297226,0.0,-2.366432,1.000000
1,부산,8111,2013,12,7,0.166124,-0.113291,0.0,-2.366432,1.000000
2,대구,8080,2013,12,5,0.164066,-0.123782,0.0,-2.366432,1.000000
3,인천,10204,2013,12,11,0.305093,0.595005,0.0,-2.366432,1.000000
4,광주,6098,2013,12,4,0.032468,-0.794515,0.0,-2.366432,1.000000
...,...,...,...,...,...,...,...,...,...,...
352,전북,6580,2015,8,13,0.064471,-0.631400,1.0,1.183216,0.636364
353,전남,6289,2015,8,12,0.045150,-0.729878,1.0,1.183216,0.636364
354,경북,7037,2015,8,3,0.094814,-0.476746,1.0,1.183216,0.636364
355,경남,7665,2015,8,2,0.136512,-0.264223,1.0,1.183216,0.636364


In [88]:
month = np.array(df['월']).reshape(-1, 1)
scaler_month = StandardScaler()
scaler_month.fit(month)
df['월n'] = scaler_month.transform(month)
df

,지역명,평당분양가격,연도,월,지역명2,평당분양가격s,평당분양가격n,연도s,연도n,월s,월n
0,서울,18189,2013,12,8,0.835270,3.297226,0.0,-2.366432,1.000000,1.767475
1,부산,8111,2013,12,7,0.166124,-0.113291,0.0,-2.366432,1.000000,1.767475
2,대구,8080,2013,12,5,0.164066,-0.123782,0.0,-2.366432,1.000000,1.767475
3,인천,10204,2013,12,11,0.305093,0.595005,0.0,-2.366432,1.000000,1.767475
4,광주,6098,2013,12,4,0.032468,-0.794515,0.0,-2.366432,1.000000,1.767475
...,...,...,...,...,...,...,...,...,...,...,...
352,전북,6580,2015,8,13,0.064471,-0.631400,1.0,1.183216,0.636364,0.589158
353,전남,6289,2015,8,12,0.045150,-0.729878,1.0,1.183216,0.636364,0.589158
354,경북,7037,2015,8,3,0.094814,-0.476746,1.0,1.183216,0.636364,0.589158
355,경남,7665,2015,8,2,0.136512,-0.264223,1.0,1.183216,0.636364,0.589158


```
지역명2: 지역명필드는 라벨인코딩하여 추가
평당분양가격n: 평당분양가격의 정규화 스케일 조정하여 추가
평당분양가격s: 평당분양가격의 표준화 스케일 조정하여 추가
연도n: 연도의 정규화 스케일 조정하여 추가
연도s: 연도의 표준화 스케일 조정하여 추가
월n: 월의 정규화 스케일 조정하여 추가
월s: 월의 표준화 스케일 조정하여 추가
```